# Locally Optimal Block Preconditioned Conjugate Gradient (LOBPCG) method

From [wikipedia page](https://en.wikipedia.org/wiki/LOBPCG):
> LOBPCG is a matrix-free method for finding the largest (or smallest) eigenvalues and the corresponding eigenvectors of a symmetric generalized eigenvalue problem.

    A x = λ B x


I found the method following this recent discussion:
[What is the most efficient method for calculating eigenvalues and eigenvectors of large matrices?](https://www.researchgate.net/post/What-is-the-most-efficient-method-for-calculating-eigenvalues-and-eigenvectors-of-large-matrices)

In particular, Andrew Knyazev, the author of the LOBPCG, makes multiple interesting remarks.


He talks about the  LOBPCG  method:
> For example, LOBPCG works only for symmetric problems, including
> generalized, to find <5-10% of extreme eigenpairs, but whether 
> the matrix is sparse or dense is not so important.
> For a full review of options, please read this [paper from 1997](https://www.researchgate.net/publication/46619332_Templates_for_the_Solution_of_Algebraic_Eigenvalue_Problems_A_Practical_Guide).


>The already mentioned LOBPCG has many implementations. In MATLAB, including support for single precision, distributed or codistributed arrays, and tiling arrays, please see
https://www.mathworks.com/matlabcentral/fileexchange/48-locally-optimal-block-preconditioned-conjugate-gradient .
A plain C version is in https://github.com/lobpcg/blopex .
In other cases, LOBPCG is already included in the hosting package, e.g., Anasazi (Trilinos), SLEPc, hypre, SciPy, Octave, Julia, MAGMA, Pytorch, Rust, RAPIDS cuGraph, and NVIDIA AMGX. For a comprehensive review, see https://en.wikipedia.org/wiki/LOBPCG


We can find an implementation in scipy:
- https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.lobpcg.html

And in pytorch:
- https://pytorch.org/docs/stable/generated/torch.lobpcg.html

Let's work with some examples.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import scipy.linalg
from scipy.sparse import issparse, spdiags
from scipy.sparse.linalg import LinearOperator
from scipy.sparse.linalg import lobpcg

In [3]:
def generate_wishart(N=1000, T=1100):
    X = np.random.randn(T, N)
    W = X.T @ X / T
    return W, X

In [4]:
rng = np.random.default_rng()

In [5]:
N = 5000
T = 100

In [6]:
W, X = generate_wishart(N, T)
X.shape

(100, 5000)

In [7]:
X0 = rng.random((N, 3))

In [32]:
%time ev, X1 = lobpcg(W, X0)
ev

CPU times: user 1.58 s, sys: 173 ms, total: 1.76 s
Wall time: 272 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.00316472 0.00520395 0.11514184]
not reaching the requested tolerance 0.00015811388300841897.


array([64.51746134, 64.33158269, 63.08495533])

Let's look if we gain in speed if we use the warm start mechanism of LOBPCG:

In [33]:
for i in range(5):
    %time ev, X0 = lobpcg(W, X0)
ev

<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.00316472 0.00520395 0.11514184]
not reaching the requested tolerance 0.00015811388300841897.
<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[8.77867972e-05 3.65462872e-05 8.29404581e-04]
not reaching the requested tolerance 0.00015811388300841897.


CPU times: user 1.53 s, sys: 186 ms, total: 1.72 s
Wall time: 284 ms
CPU times: user 964 ms, sys: 132 ms, total: 1.1 s
Wall time: 186 ms
CPU times: user 230 ms, sys: 38.3 ms, total: 268 ms
Wall time: 56.6 ms
CPU times: user 59.6 ms, sys: 5.76 ms, total: 65.4 ms
Wall time: 11.8 ms
CPU times: user 58.2 ms, sys: 9.75 ms, total: 67.9 ms
Wall time: 14.7 ms


array([64.51746262, 64.33158568, 63.08765921])

Now Let's try to exploit the structure of the matrix and the fact that in our current example the data matrix $X$ is very thin.

In [34]:
A = LinearOperator(matvec=lambda v: W @ v, shape=(N, N), dtype=float)

In [35]:
X0 = rng.random((N, 3))
%time ev, X1 = lobpcg(A, X0)
ev

CPU times: user 3.06 s, sys: 376 ms, total: 3.44 s
Wall time: 504 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.00361279 0.00648843 0.08669973]
not reaching the requested tolerance 0.00015811388300841897.


array([64.51746138, 64.33158166, 63.08625721])

In [36]:
A = LinearOperator(
    matvec=lambda v: (X.T @ (X @ v)) / T,
    # rmatvec=lambda v: (X.T @ (X @ v))/T,
    # matmat=lambda v: (X.T @ (X @ v))/T,
    shape=(N, N),
    dtype=float,
)

In [37]:
X0 = rng.random((N, 3))
%time ev, X1 = lobpcg(A, X0)
ev

CPU times: user 201 ms, sys: 84 ms, total: 285 ms
Wall time: 73.5 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.00604918 0.01301494 0.1429378 ]
not reaching the requested tolerance 0.00015811388300841897.


array([64.51745771, 64.33156625, 63.08381867])

In [38]:
for i in range(6):
    %time ev, X0 = lobpcg(A, X0)
ev

CPU times: user 217 ms, sys: 106 ms, total: 323 ms
Wall time: 54.2 ms
CPU times: user 138 ms, sys: 69.5 ms, total: 208 ms
Wall time: 37.9 ms
CPU times: user 27.2 ms, sys: 14.1 ms, total: 41.3 ms
Wall time: 8.7 ms
CPU times: user 7.04 ms, sys: 3.39 ms, total: 10.4 ms
Wall time: 1.66 ms
CPU times: user 8.06 ms, sys: 4.06 ms, total: 12.1 ms
Wall time: 2.15 ms
CPU times: user 7.56 ms, sys: 4.04 ms, total: 11.6 ms
Wall time: 1.78 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.00604918 0.01301494 0.1429378 ]
not reaching the requested tolerance 0.00015811388300841897.
<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[3.54576991e-05 7.61459513e-05 6.76479286e-04]
not reaching the requested tolerance 0.00015811388300841897.


array([64.51746262, 64.33158568, 63.08765921])

We end with a very small execution time!

Can we use preconditionner to speedup again things

Initial guess for eigenvectors, should have linearly independent
columns. Column dimension = number of requested eigenvalues.


Preconditioner in the inverse of A in this example:


In [22]:
invA = spdiags([1.0 / (X**2).mean(0)], 0, N, N)

The preconditiner must be defined by a function.

The argument x of the preconditioner function is a matrix inside `lobpcg`, thus the use of matrix-matrix product ``@``.

The preconditioner function is passed to lobpcg as a `LinearOperator`:


In [23]:
M = LinearOperator(matvec=lambda v: invA @ v, shape=(N, N), dtype=float)

Let us now solve the eigenvalue problem for the matrix A:


In [24]:
rng = np.random.default_rng()
X0 = rng.random((N, 3))

In [25]:
%time ev, X1 = lobpcg(A, X0, M=M, largest=True)
ev

CPU times: user 211 ms, sys: 99 ms, total: 310 ms
Wall time: 69.9 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.66552245 0.94438162 1.23658475]
not reaching the requested tolerance 0.00015811388300841897.


array([64.4363801 , 64.26032347, 62.92400588])

In [26]:
for i in range(6):
    %time ev, X0 = lobpcg(A, X0, M=M, largest=True)
X0

CPU times: user 230 ms, sys: 108 ms, total: 338 ms
Wall time: 58.8 ms
CPU times: user 234 ms, sys: 107 ms, total: 341 ms
Wall time: 56.5 ms
CPU times: user 163 ms, sys: 82 ms, total: 245 ms
Wall time: 52.9 ms
CPU times: user 70.3 ms, sys: 40.2 ms, total: 110 ms
Wall time: 20.7 ms
CPU times: user 7.39 ms, sys: 3.8 ms, total: 11.2 ms
Wall time: 2.19 ms
CPU times: user 13.5 ms, sys: 4.44 ms, total: 17.9 ms
Wall time: 6.25 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.66552245 0.94438162 1.23658475]
not reaching the requested tolerance 0.00015811388300841897.
<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.00210415 0.00163139 0.02752275]
not reaching the requested tolerance 0.00015811388300841897.
<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[0.00012043 0.00010094 0.00136726]
not reaching the requested tolerance 0.00015811388300841897.


array([[ 0.01779802, -0.01164631, -0.00203599],
       [ 0.03196724,  0.01505311, -0.00678067],
       [-0.01258953, -0.00896628, -0.01156641],
       ...,
       [ 0.00345719,  0.03078773,  0.01879537],
       [ 0.00399709, -0.01288702,  0.00355878],
       [-0.00343277,  0.00610764, -0.01910672]])

Constraints:

Note that the vectors passed in Y are the eigenvectors of the 3 smallest
    eigenvalues. The results returned are orthogonal to those.

The matrix constraint should be used when solving the eigen problem while embending the problem in a larger dimensional space.

In [10]:
Y = np.eye(N, 5)

In [28]:
rng = np.random.default_rng()
X0 = rng.random((N, 3))

In [29]:
%time ev, X1 = lobpcg(A, X0, Y=Y, M=M, largest=True)
ev

CPU times: user 201 ms, sys: 84.3 ms, total: 285 ms
Wall time: 72.9 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[2.28404938 1.83044535 1.79433101]
not reaching the requested tolerance 0.00015811388300841897.


array([64.41076404, 64.28373849, 62.89652485])

In [12]:
for i in range(6):
    %time ev, X0 = lobpcg(A, X0, Y=Y, M=M, largest=True)
X0

<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[1.38228946 2.33042469 0.62485752]
not reaching the requested tolerance 0.00015811388300841897.


CPU times: user 1.41 s, sys: 224 ms, total: 1.63 s
Wall time: 317 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[1.38287338 2.33878094 0.62359243]
not reaching the requested tolerance 0.00015811388300841897.


CPU times: user 1.39 s, sys: 197 ms, total: 1.59 s
Wall time: 312 ms


<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[1.38287266 2.33877924 0.62359243]
not reaching the requested tolerance 0.00015811388300841897.


CPU times: user 1.5 s, sys: 207 ms, total: 1.71 s
Wall time: 291 ms
CPU times: user 1.55 s, sys: 166 ms, total: 1.71 s
Wall time: 283 ms
CPU times: user 1.58 s, sys: 188 ms, total: 1.77 s
Wall time: 280 ms
CPU times: user 1.62 s, sys: 174 ms, total: 1.8 s
Wall time: 266 ms


array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       ...,
       [-0.00743823, -0.00704139,  0.01313926],
       [ 0.00525484, -0.01749841, -0.0073815 ],
       [ 0.00071932,  0.01216465,  0.00130925]])

In [13]:
for i in range(6):
    %time ev, X0 = lobpcg(W, X0, Y=Y, largest=True)
X0

<timed exec>:1: UserWarning: Exited at iteration 20 with accuracies 
[1.38287266 2.33877924 0.62359243]
not reaching the requested tolerance 0.00015811388300841897.


CPU times: user 1.54 s, sys: 165 ms, total: 1.7 s
Wall time: 282 ms
CPU times: user 1.6 s, sys: 190 ms, total: 1.79 s
Wall time: 279 ms
CPU times: user 1.64 s, sys: 184 ms, total: 1.83 s
Wall time: 266 ms
CPU times: user 1.58 s, sys: 178 ms, total: 1.76 s
Wall time: 280 ms
CPU times: user 1.59 s, sys: 203 ms, total: 1.79 s
Wall time: 276 ms
CPU times: user 1.6 s, sys: 197 ms, total: 1.8 s
Wall time: 277 ms


array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ],
       ...,
       [ 0.00743823, -0.00704139, -0.01313926],
       [-0.00525484, -0.01749841,  0.0073815 ],
       [-0.00071932,  0.01216465, -0.00130925]])

When constraints are specified, then warm start does not matter anymore.